# Request vercel API
![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Import library

In [ ]:
from github import Github
import os
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

## Bước 1: Chọn repo trên Github

### Kết nối tới repo

In [ ]:
g = Github("")
repo_owner = 'vercel'
repo_name = 'vercel'

### Lấy thông tin repo

In [ ]:
repo = g.get_repo(f"{repo_owner}/{repo_name}")

## Bước 2: Thu thập dữ liệu

##### Lấy danh sách commits

In [ ]:
prs = repo.get_commits()

### Tạo danh sách lưu chữ thông tin về commits

In [ ]:
commit_data = []

for pr in prs:
    print(pr.last_modified_datetime)
    commit_info = {
        "sha": pr.sha,
        "author": pr.commit.author.name,
        "author_date": pr.commit.author.date,
        "committer": pr.commit.committer.name,
        "committer_date": pr.commit.committer.date,
        "message": pr.commit.message,
    }
    commit_data.append(commit_info)

## Bước 3: Làm sạch vè tiền xử lý dữ liệu

### Chuyển đổi danh sách thành DataFrame và lưu vào data

In [ ]:
data_dir = '../data'
csv_file = os.path.join(data_dir, 'commits_data.csv')

df_commits = pd.DataFrame(commit_data)
df_commits.to_csv(csv_file, index=False)

print(df_commits.head())

### Làm sạch dữ liệu

##### Kiểm tra thông tin còn thiếu

In [ ]:
print(df_commits.isnull().sum())

##### Xử lý dữ liệu thiếu

In [ ]:
df_commits = df_commits.dropna()

##### Kiểm tra lại dữ liệu

In [ ]:
print(df_commits.isnull().sum())

##### Kiểm tra dữ liệu trùng lặp

In [ ]:
print(df_commits.duplicated().sum())

##### Loại bỏ các hàng trùng lặp

In [ ]:
df_commits = df_commits.drop_duplicates()

##### Kiểm tra và lưu lại dữ liệu sau khi làm sạch

In [ ]:
print(df_commits.duplicated().sum())

csv_file = os.path.join(data_dir, 'commits_data_cleaned.csv')
df_commits.to_csv(csv_file, index=False)

### Tiền xử lý dữ liệu

#### Lọc dữ liệu để chỉ bao gồm commits trong năm 2023

In [ ]:
df_2023 = pd.DataFrame(df_commits[df_commits['author_date'].dt.year == 2023])

print(df_2023.head())

##### Chuyển đổi cột 'author_date' và 'committer_date' thành định dạng datetime

In [ ]:
df_2023['author_date'] = pd.to_datetime(df_2023['author_date'])
df_2023['committer_date'] = pd.to_datetime(df_2023['committer_date'])

print(df_2023.dtypes)

##### Thêm cột 'year', 'month' để phân tích theo thời gian

In [ ]:
df_2023['year'] = df_2023['author_date'].dt.year
df_2023['month'] = df_2023['author_date'].dt.month


##### Lưu trữ lại DataFrame sau khi tiền xử lý

In [ ]:
csv_file = os.path.join(data_dir, 'commits_data_preprocessed.csv')
df_2023.to_csv(csv_file, index=False)
print(df_2023.head())

## Bước 4 & 5: Phân tích và trực quan hoá dữ liệu

In [ ]:
plots_dir = '../plots'

### 1. Phân tích số lượng commits theo tháng trong năm 2023

##### Xác định số lượng commits theo tháng

In [ ]:
commits_per_month_2023 = df_2023.groupby(df_2023['author_date'].dt.month).size()
print(commits_per_month_2023)

##### Trực quan hóa số lượng commits theo tháng trong năm 2023

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(commits_per_month_2023.index, commits_per_month_2023.values, marker='o')
plt.title('Number of Commits per Month in 2023')
plt.xlabel('Month')
plt.ylabel('Number of Commits')
plt.xticks(commits_per_month_2023.index)
plt.grid(True)
plot_file = os.path.join(plots_dir, 'commits_per_month.png')
plt.savefig(plot_file)
plt.show()
plt.close()

### 2. Phân tích số lượng commits theo ngày trong tuần trong năm 2023

##### Tạo cột 'day_of_week' để lưu thông tin ngày trong tuần (0: Monday, 6: Sunday)

In [ ]:
df_2023['day_of_week'] = df_2023['author_date'].dt.dayofweek

##### Tính số lượng commits và đặt tên cho các ngày trong tuần

In [ ]:
commits_per_day_of_week = df_2023['day_of_week'].value_counts().sort_index()
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
commits_per_day_of_week.index = day_names

##### Vẽ biểu đồ số lượng commits theo ngày trong tuần

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(commits_per_day_of_week.index, commits_per_day_of_week.values, color='skyblue')
plt.title('Number of Commits per Day of the Week in 2023')
plt.xlabel('Day of the Week')
plt.ylabel('Number of Commits')
plt.grid(axis='y')
plot_file = os.path.join(plots_dir, 'commits_per_day.png')
plt.savefig(plot_file)
plt.show()
plt.close()

### 3. Phân tích số lượng commits theo giờ trong ngày trong năm 2023

##### Tạo cột 'hour_of_day' để lưu thông tin giờ trong ngày (0-23)

In [ ]:
df_2023['hour_of_day'] = df_2023['author_date'].dt.hour

##### Tính số lượng commits cho mỗi giờ trong ngày

In [ ]:
commits_per_hour = df_2023['hour_of_day'].value_counts().sort_index()

##### Vẽ biểu đồ số lượng commits theo giờ trong ngày

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(commits_per_hour.index, commits_per_hour.values, color='skyblue')
plt.title('Number of Commits per Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Commits')
plt.xticks(range(24))
plt.grid(axis='y')
plot_file = os.path.join(plots_dir, 'commits_per_hour.png')
plt.savefig(plot_file)
plt.show()
plt.close()

### 4. Phân tích tần suất commits của các tác giả trong năm 2023

##### Xác định tần suất commits của các tác giả trong năm 2023

In [ ]:
commits_per_author_2023 = df_2023['author'].value_counts()
print(commits_per_author_2023.head(10))

##### Trực quan hóa tần suất commits của các tác giả trong năm 2023

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(commits_per_author_2023.index[:10], commits_per_author_2023.values[:10], color='skyblue')
plt.title('Top 10 Authors by Number of Commits in 2023')
plt.xlabel('Author')
plt.ylabel('Number of Commits')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plot_file = os.path.join(plots_dir, 'top_author_commits.png')
plt.savefig(plot_file)
plt.show()
plt.close()

### 5. Phân tích tần suất các từ khoá quan trọng

##### Tạo một danh sách các từ khóa quan trọng cần phân tích

In [ ]:
keywords = ["fix", "add", "update", "remove", "rename", "refactor", "test", "merge", "release", "build"]

##### Đếm tần suất các từ khóa trong commit message

In [ ]:
def count_keywords(message, keywords):
    message = message.lower()
    return {keyword: message.count(keyword) for keyword in keywords}

df_2023['keyword_counts'] = df_2023['message'].apply(lambda x: count_keywords(x, keywords))

##### Tổng hợp tần suất các từ khóa

In [ ]:
keyword_totals = Counter()
for counts in df_2023['keyword_counts']:
    keyword_totals.update(counts)

##### Trực quan hóa tần suất các từ khóa

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(keyword_totals.keys(), keyword_totals.values(), color='skyblue')
plt.title('Frequency of Keywords in Commit Messages')
plt.xlabel('Keyword')
plt.ylabel('Frequency')
plot_file = os.path.join(plots_dir, 'frequency_keywords.png')
plt.savefig(plot_file)
plt.show()
plt.close()


### 6. Phần tích độ dài của các message

##### Tính độ dài của mỗi commit message và độ dài trung bình

In [ ]:
df_2023['message_length'] = df_2023['message'].apply(len)

average_length = df_2023['message_length'].mean()
print(f"Average commit message length: {average_length:.2f} characters")

##### Trực quan hóa phân phối độ dài của commit message

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df_2023['message_length'], bins=30, color='skyblue', edgecolor='black')
plt.axvline(average_length, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {average_length:.2f} chars')
plt.title('Distribution of Commit Message Lengths')
plt.xlabel('Message Length (characters)')
plt.ylabel('Frequency')
plt.legend()
plot_file = os.path.join(plots_dir, 'commits_lengths.png')
plt.savefig(plot_file)
plt.show()
plt.close()


### 7. Phân tích xu hướng nội dung commit theo tháng

##### Chuyển cột 'author_date' sang chỉ chứa tháng và năm

In [ ]:
df_2023['month_year'] = df_2023['author_date'].dt.strftime('%Y-%m')

##### Tính tần suất từ khóa theo tháng

In [ ]:
keyword_trends = {keyword: [] for keyword in keywords}
dates = sorted(df_2023['month_year'].unique())

for date in dates:
    df_month = df_2023[df_2023['month_year'] == date]
    monthly_counts = Counter()
    for counts in df_month['keyword_counts']:
        monthly_counts.update(counts)
    for keyword in keywords:
        keyword_trends[keyword].append(monthly_counts[keyword])

##### Trực quan hóa xu hướng từ khóa theo tháng

In [ ]:
plt.figure(figsize=(12, 6))
for keyword in keywords:
    plt.plot(dates, keyword_trends[keyword], marker='o', label=keyword)

plt.title('Keyword Trends in Commit Messages Over Time')
plt.xlabel('Time (Month-Year)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plot_file = os.path.join(plots_dir, 'keywords_trends_commits.png')
plt.savefig(plot_file)
plt.show()
plt.close()
